In [1]:
import numpy as np
import tensorflow as tf
import cPickle as pickle
import sklearn.metrics as sm

In [2]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [3]:
def confusion_matrix(predictions, labels):
    pred = np.argmax(predictions, 1)
    actual = np.argmax(labels, 1)
    
    print "Actuals ", np.bincount(actual)
    print "Predictions ", np.bincount(pred)
    
    print "Accuracy ", 100.0 * np.sum(pred == actual) / pred.shape[0]
    print "Two class confusion matrix"
    p = (pred != 0)
    a = (actual != 0)
    print sm.confusion_matrix(p, a)

In [4]:
pickle_file = 'badged_images.pickle'
num_labels = 10

def one_hot_encoding(labels):
  return (np.arange(num_labels) == labels[:,None]).astype(np.float32)

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = one_hot_encoding(save['train_labels'])
  valid_dataset = save['valid_dataset']
  valid_labels = one_hot_encoding(save['valid_labels'])
  test_dataset = save['test_dataset']
  test_labels = one_hot_encoding(save['test_labels'])
  del save  # hint to help gc free up memory

  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

print "Validation baseline accuracy"
confusion_matrix(one_hot_encoding(np.zeros(valid_labels.shape[0])), valid_labels)

print "Test baseline accuracy"
confusion_matrix(one_hot_encoding(np.zeros(test_labels.shape[0])), test_labels)

('Training set', (2000, 300, 256, 3), (2000, 10))
('Validation set', (250, 300, 256, 3), (250, 10))
('Test set', (250, 300, 256, 3), (250, 10))
Validation baseline accuracy
Actuals  [217   3   3   3   2   7   6   3   3   3]
Predictions  [250]
Accuracy  86.8
Two class confusion matrix
[[217  33]
 [  0   0]]
Test baseline accuracy
Actuals  [223   3   5   3   4   4   1   4   0   3]
Predictions  [250]
Accuracy  89.2
Two class confusion matrix
[[223  27]
 [  0   0]]


In [5]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [8]:
batch_size = 50
patch_size = 5
depth = 16
num_hidden = 64

image_height = train_dataset.shape[1]
image_width = train_dataset.shape[2]
num_channels = train_dataset.shape[3]

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_height, image_width, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_height // 4 * image_width // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    h_conv1 = tf.nn.relu(conv2d(data, layer1_weights) + layer1_biases)
    h_pool1 = max_pool_2x2(h_conv1) 
    
    h_conv2 = tf.nn.relu(conv2d(h_pool1, layer2_weights) + layer2_biases)
    h_pool2 = max_pool_2x2(h_conv2)

    shape = h_pool2.get_shape().as_list()
    reshape = tf.reshape(h_pool2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [9]:
%%time
num_steps = 501

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
    
      valid_preds = valid_prediction.eval()
      print('Validation accuracy: %.1f%%' % accuracy(valid_preds, valid_labels))
      confusion_matrix(valid_preds, valid_labels)

  test_preds = test_prediction.eval()
  print('Test accuracy: %.1f%%' % accuracy(test_preds, test_labels))
  confusion_matrix(test_preds, test_labels)

Initialized
Minibatch loss at step 0: 2498.747070
Minibatch accuracy: 8.0%
Validation accuracy: 86.8%
Actuals  [217   3   3   3   2   7   6   3   3   3]
Predictions  [250]
Accuracy  86.8
Two class confusion matrix
[[217  33]
 [  0   0]]
Logits  (50, 10) 0
Layer1 weights (5, 5, 3, 16) 0
Minibatch loss at step 50: nan
Minibatch accuracy: 92.0%
Validation accuracy: 86.8%
Actuals  [217   3   3   3   2   7   6   3   3   3]
Predictions  [250]
Accuracy  86.8
Two class confusion matrix
[[217  33]
 [  0   0]]
Logits  (50, 10) 500
Layer1 weights (5, 5, 3, 16) 1200
Minibatch loss at step 100: nan
Minibatch accuracy: 84.0%
Validation accuracy: 86.8%
Actuals  [217   3   3   3   2   7   6   3   3   3]
Predictions  [250]
Accuracy  86.8
Two class confusion matrix
[[217  33]
 [  0   0]]
Logits  (50, 10) 500
Layer1 weights (5, 5, 3, 16) 1200


KeyboardInterrupt: 